# 租稅法總論稿件合併

這個 notebook 會將 `_Material/1121稿/Filtered_租稅法總論` 中的逐次講義整合成單一的 Word 檔案，並依據每份檔案的 YAML 前言建立段落標題。


In [13]:
# 如環境尚未安裝相關套件，可以先執行這個區塊。
%pip install -q python-docx pyyaml


Note: you may need to restart the kernel to use updated packages.


In [14]:
from pathlib import Path
import re
import yaml
from docx import Document
from docx.oxml.ns import qn

BASE_DIR = Path.cwd()
possible_roots = [BASE_DIR, BASE_DIR / 'work'] + list(BASE_DIR.parents) + list((BASE_DIR / 'work').parents)


def resolve_existing(relative: Path) -> Path | None:
    candidate = Path(relative)
    if candidate.is_absolute():
        return candidate if candidate.exists() else None
    for base in possible_roots:
        target = (Path(base) / candidate).resolve()
        if target.exists():
            return target
    return None


SOURCE_CANDIDATES = [
    Path('mkdocs/My_Notes/Filtered_租稅法總論'),
    Path('_Material/1121稿/Filtered_租稅法總論'),
]

SOURCE_DIR = None
SOURCE_RELATIVE = None
for candidate in SOURCE_CANDIDATES:
    resolved = resolve_existing(candidate)
    if resolved is not None:
        SOURCE_RELATIVE = candidate
        SOURCE_DIR = resolved
        break

if SOURCE_DIR is None:
    joined = '\n - '.join(str(c) for c in SOURCE_CANDIDATES)
    raise FileNotFoundError('找不到講義資料夾，已嘗試:- ' + joined)

OUTPUT_PATH = SOURCE_DIR / '租稅法總論_合併.docx'

print(f'Working directory: {BASE_DIR}')
print(f'使用資料夾     : {SOURCE_RELATIVE}')
print(f'Source directory : {SOURCE_DIR}')
print(f'Output file      : {OUTPUT_PATH}')


Working directory: /home/jovyan
使用資料夾     : mkdocs/My_Notes/Filtered_租稅法總論
Source directory : /home/jovyan/work/mkdocs/My_Notes/Filtered_租稅法總論
Output file      : /home/jovyan/work/mkdocs/My_Notes/Filtered_租稅法總論/租稅法總論_合併.docx


In [15]:
from pathlib import Path
import re
import yaml
from docx import Document
from docx.oxml.ns import qn

# === 全域設定 ===
PREFERRED_METADATA_KEYS = ['課程', '日期', '周次', '節次']
SUPPORTED_EXTENSIONS = ('.md',)
EAST_ASIA_FONT = '標楷體'
HEADING_PATTERN = re.compile(r'^(#{1,6})\s+(.*)$')
SHOW_METADATA_AFTER_HEADING = False

# === YAML front matter 解析 ===
def parse_front_matter(raw_text):
    """
    解析 Markdown 檔案開頭的 front matter。
    支援全形冒號、自動去除註解與空白，
    若 YAML 格式無效仍能逐行正確解析。
    並移除所有含「課程」字樣的欄位。
    """
    raw_text = raw_text.lstrip('﻿')
    pattern = r'^---\s*\n(.*?)\n---\s*\n?'
    match = re.match(pattern, raw_text, flags=re.DOTALL)
    if not match:
        return {}, raw_text.strip()

    meta_block = match.group(1)
    meta = {}

    # 逐行手動解析，保證鍵值對被正確拆分
    for line in meta_block.splitlines():
        line = line.strip()
        if not line or line.startswith('#'):
            continue
        # 將全形冒號統一為半形
        line = line.replace('：', ':', 1)
        if ':' not in line:
            continue
        key, value = line.split(':', 1)
        key = key.strip().replace(':', '').replace('：', '')
        value = value.strip()
        # 移除任何含「課程」的欄位
        if '課程' in key:
            continue
        meta[key] = value

    body = raw_text[match.end():].lstrip('\r\n')
    return meta, body


# === 字型設定 ===
def apply_run_font(run):
    run.font.name = EAST_ASIA_FONT
    r = run._element
    rPr = getattr(r, "rPr", None)
    if rPr is None or len(rPr) == 0:
        rPr = r.get_or_add_rPr()
    rFonts = getattr(rPr, "rFonts", None)
    if rFonts is None or len(rFonts) == 0:
        rFonts = rPr.get_or_add_rFonts()
    rFonts.set(qn('w:eastAsia'), EAST_ASIA_FONT)
    rFonts.set(qn('w:ascii'), EAST_ASIA_FONT)
    rFonts.set(qn('w:hAnsi'), EAST_ASIA_FONT)


# === 標題生成 ===
def build_heading(metadata, fallback_label):
    """生成講次小標（已保證不含課程欄位）。"""
    if not metadata:
        return fallback_label

    # 保險：再過濾一次
    metadata = {
        k: v
        for k, v in metadata.items()
        if '課程' not in str(k).replace('：', '').replace(':', '').strip()
    }

    parts = []
    if metadata.get('周次'):
        w = str(metadata['周次'])
        parts.append(f"第{w}週" if not w.startswith('第') else w)
    if metadata.get('節次'):
        s = str(metadata['節次'])
        parts.append(f"第{s}節" if not s.startswith('第') else s)
    if metadata.get('日期'):
        parts.append(str(metadata['日期']))

    if not parts:
        extras = [str(v).strip() for v in metadata.values() if v and str(v).strip()]
        return '｜'.join(extras) if extras else fallback_label
    return '｜'.join(parts)


# === 解析 Markdown ===
def extract_blocks(body_text):
    blocks, buf = [], []
    for line in body_text.splitlines():
        if not line.strip():
            if buf:
                blocks.append(('paragraph', '\n'.join(buf).rstrip()))
                buf = []
            continue
        m = HEADING_PATTERN.match(line.rstrip())
        if m:
            if buf:
                blocks.append(('paragraph', '\n'.join(buf).rstrip()))
                buf = []
            blocks.append(('heading', len(m.group(1)), m.group(2).strip()))
        else:
            buf.append(line.rstrip())
    if buf:
        blocks.append(('paragraph', '\n'.join(buf).rstrip()))
    return blocks


# === 收集所有條目 ===
def collect_entries(source_dir):
    """讀取 Markdown 並再次過濾所有含課程欄位。"""
    if not source_dir.exists():
        raise FileNotFoundError(f'找不到資料夾: {source_dir}')

    files = sorted([p for ext in SUPPORTED_EXTENSIONS for p in source_dir.glob(f'*{ext}')])
    if not files:
        raise RuntimeError('資料夾中沒有找到任何支援的檔案 (.md)')

    entries = []
    for path in files:
        raw = path.read_text(encoding='utf-8')
        meta, body = parse_front_matter(raw)

        # **再次清理**
        meta = {
            k: v
            for k, v in meta.items()
            if '課程' not in str(k).replace('：', '').replace(':', '').strip()
        }

        blocks = extract_blocks(body)
        entries.append({'path': path, 'meta': meta or {}, 'blocks': blocks})
    return entries


# === Word 輔助函式 ===
def add_text_paragraph(doc, text):
    p = doc.add_paragraph()
    for i, line in enumerate(text.split('\n')):
        if i > 0:
            p.add_run().add_break()
        run = p.add_run(line)
        apply_run_font(run)

def add_heading(doc, text, level):
    h = doc.add_heading(text, level=level)
    for run in h.runs:
        apply_run_font(run)

def add_metadata_block(doc, metadata):
    """美觀列印 metadata（已確保無課程鍵）。"""
    metadata = {
        k: v
        for k, v in metadata.items()
        if '課程' not in str(k).replace('：', '').replace(':', '').strip()
    }
    if not metadata:
        return
    items = []
    for k in PREFERRED_METADATA_KEYS:
        if k in metadata:
            items.append((k, metadata[k]))
    for k, v in metadata.items():
        if k in PREFERRED_METADATA_KEYS:
            continue
        items.append((k, v))
    for k, v in items:
        para = doc.add_paragraph(f"{k}: {v}", style='List Bullet')
        for run in para.runs:
            apply_run_font(run)
    doc.add_paragraph('')


# === 文件主流程 ===
def make_document(entries, output_path):
    doc = Document()
    add_heading(doc, '租稅法總論講義彙編', 1)
    doc.add_page_break()
    # print(entries)

    for i, entry in enumerate(entries):
        heading = build_heading(entry['meta'], entry['path'].stem)
        add_heading(doc, heading, 2)

        if SHOW_METADATA_AFTER_HEADING:
            add_metadata_block(doc, entry['meta'])

        for b in entry['blocks']:
            if b[0] == 'heading':
                lvl, text = b[1], b[2]
                add_heading(doc, text, min(lvl + 2, 9))
            else:
                add_text_paragraph(doc, b[1])

        if i != len(entries) - 1:
            doc.add_page_break()

    output_path.parent.mkdir(parents=True, exist_ok=True)
    doc.save(output_path)
    return output_path


# === 主控 ===
def compile_word_document():
    entries = collect_entries(SOURCE_DIR)
    path = make_document(entries, OUTPUT_PATH)
    print(f'完成：{path}')

# === 執行 ===
compile_word_document()

完成：/home/jovyan/work/mkdocs/My_Notes/Filtered_租稅法總論/租稅法總論_合併.docx
